# Sample SDK

## Pandas HTML + BeautifulSoup



In [88]:
!pip install requests-random-user-agent

In [89]:
import multiprocessing
multiprocessing.cpu_count()

2

In [90]:

import requests
import requests_random_user_agent
#s = requests.Session()
#print(s.headers['User-Agent'])

# Without a session
resp = requests.get('https://httpbin.org/user-agent')
print(resp.json()['user-agent'])

Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.2 Safari/605.1.15


In [91]:
import concurrent.futures
import requests
import requests_random_user_agent
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

def _download_SDK(url):

  df_tempSDK = pd.DataFrame()

  rq = requests.get(url)
  
  if rq.status_code == 404 or rq.status_code == 403: ## Handle more error codes...
    exit

  main_data = rq.text

  # dataset from table
  df_tempSDK = pd.read_html(main_data, index_col=0)[0]

  main_soup = BeautifulSoup(main_data, 'html.parser')
  
  main_names = main_soup.find_all('tr')[1:245]

  list_urlSDK = []

  # Head url for meta_url
  head_Url= 'https://www.programmableweb.com'

  for row in main_names:
       text = row.find_all('td')[0]      
       list_urlSDK.append( (head_Url + str(text).partition('<a href="')[2].partition('">')[0])) # Meta URL 


  df_tempSDK['Meta_Url'] = list_urlSDK

  return df_tempSDK


def download_SDK(bulk_urls, num_Splits):
  df_temp = pd.DataFrame()

  # Split urls
  urls_splited  = np.array_split(bulk_urls, num_Splits) # max workers

  tasks = []

  for split in range(len(urls_splited)):
    with concurrent.futures.ThreadPoolExecutor(max_workers = len(urls_splited)) as executor:
      for url in urls_splited[split]:    
        tasks.append(executor.submit(_download_sourceCode, url))
       ## mb call with bulk urls instead of one by one
  # Union
  for result in tasks:
    df_temp = df_temp.append(result.result())

  return df_temp


In [126]:
SDK_urls=[]

for i in range(10): ## web pages?.. 615+1
    main_url = 'https://www.programmableweb.com/category/all/sdk&page=' + str(i) ## parametrizar + comprobar que hay datos
    SDK_urls.append(main_url)

In [127]:
#SDK_urls
df_SDK = pd.DataFrame()
df_SDK = download_SDK(SDK_urls,5) # Num workers
df_SDK.drop('Submitted', inplace=True, axis=1)
df_SDK

,Related APIs,Category,Meta_Url
SDK Name,,,
WaifuAI Chatbot NodeJS SDK,WaifuAI,Chat,https://www.programmableweb.com/sdk/waifuai-ch...
SecuGen Web JavaScript SDK,SecuGen Web,Biometrics,https://www.programmableweb.com/sdk/secugen-we...
Neuro AI SDK,Neuro AI,Machine Learning,https://www.programmableweb.com/sdk/neuro-ai-sdk
CurrencyAPI NodeJS SDK,CurrencyAPI,Currency,https://www.programmableweb.com/sdk/currencyap...
CurrencyAPI PHP SDK,CurrencyAPI,Currency,https://www.programmableweb.com/sdk/currencyap...


## Meta URL Processing

In [128]:
# Creates new columns
df_SDK['Description'] = ""

df_SDK['Related APIs'] = ""
df_SDK['Languages'] = ""
df_SDK['Provider'] = ""
df_SDK['URL'] = ""
df_SDK['Repository'] = ""


In [129]:
def _download_meta_url(df_source):

  for i in range(len(df_source)):

    meta_url = df_source['Meta_Url'][i]
    rq = requests.get(meta_url)

    if rq.status_code == 404 or rq.status_code == 403: ## Handle more error codes...
      continue

    meta_data = rq.text

    meta_soup = BeautifulSoup(meta_data, 'html.parser')

    # Update Description from the meta url
    meta_description = str(meta_soup.find('div', class_='tabs-header_description')).partition('">')[2].partition('</')[0]

    #print(meta_description)
    df_source['Description'][i] = meta_description 

    meta_specs = meta_soup.find('div', class_='section specs')

    #print(meta_specs)

    for lab in meta_specs.select("label"):   

      # Search for Related APIs
      if (lab.text.lower().find("related apis") > -1):
          #print(lab.text + ": " + lab.find_next_sibling().text)
          df_source['Related APIs'][i] =   lab.find_next_sibling().text

      # Search for Languages
      if (lab.text.lower().find("languages") > -1):
          #print(lab.text + ": " + lab.find_next_sibling().text)
          df_source['Languages'][i] =   lab.find_next_sibling().text

       # Search for Categories and remplace them
      if (lab.text.lower().find("categories") > -1):
          #print(lab.text + ": " + lab.find_next_sibling().text)
          df_source['Category'][i] =   lab.find_next_sibling().text

      # Search for Provider
      if (lab.text.lower().find("sdk provider") > -1):
         #print(lab.text + ": " + lab.find_next_sibling().text)
          df_source['Provider'][i] =   lab.find_next_sibling().text

      # Search for Provider
      if (lab.text.lower().find("url") > -1):
         #print(lab.text + ": " + lab.find_next_sibling().text)
          df_source['URL'][i] =   lab.find_next_sibling().text

      # Search for Provider
      if (lab.text.lower().find("repository") > -1):
         #print(lab.text + ": " + lab.find_next_sibling().text)
          df_source['Repository'][i] =   lab.find_next_sibling().text

    #print(df_source)

  return df_source

def download_meta_url(df_sourceCode, num_Splits):
  # Split dataframe
  dtframe_splited  = np.array_split(df_sourceCode, num_Splits) # max workers == num_Splits
  tasks = []

  for split in range(len(dtframe_splited)):
    with concurrent.futures.ThreadPoolExecutor(max_workers = len(dtframe_splited)) as executor:
      # Download dataframes splited
      tasks.append(executor.submit(_download_meta_url, dtframe_splited[split]))

  df_temp = pd.DataFrame()
  # Union
  for result in tasks:
    df_temp = df_temp.append(result.result())

  return df_temp


In [ ]:
### TEST WITH GRAB 

In [132]:
df_SDK_metaData

,index


In [131]:
# Meta_Url could be used to check for updates on the source website. That uses only +600 web requests instead of +15k
# save a copy of the original dataframe to check for updates based on the meta url or other fields
df_SDK_metaData.reset_index(inplace=True)
df_SDK_metaData.drop('Meta_Url', inplace=True, axis=1)
df_SDK_metaData

# To CSV (index True 0,1,2...)
df_SDK_metaData.to_csv(r'/content/DataFrame/original_SDK_' + datetime.now().strftime('%d_%m_%Y') + '.csv', index = True, header = True)


KeyError: ignored

In [130]:
df_SDK_metaData = pd.DataFrame()
df_SDK_metaData = download_meta_url(df_SDK,5)
df_SDK_metaData.head()

KeyboardInterrupt: ignored